<a href="https://colab.research.google.com/github/JackyChen2T2/REScipe/blob/main/REScipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# for loading google drive files
from google.colab import drive
drive.mount('/content/drive')

# for data cleaning
import pandas as pd

# for tokenizing english words
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

# for the option of using GloVe representation
import torchtext
from torchtext import data
import spacy
!python -m spacy download en

In [76]:
# flags
XLSX_PATH = '/content/drive/My Drive/Colab Notebooks/Project_REScipe/data/total_data.xlsx'      # copy the path of the xlsx file in Path in Google Colab menu
XLSX_COLUMN = ['rand_int_0', 'name', 'url', 'rand_int_1', 'size', 'ingredient', 'recipe', 'rand_int_2']
ALL_DATA = False

===================== A =====================

This section loads the excel file(.xlsx) into a Pandas Dataframe.


In [82]:
# load the xlsx file from Google Drive to a Pandas Dataframe
xlsx_total_data = pd.read_excel(XLSX_PATH, header=None, names=XLSX_COLUMN)

print(xlsx_total_data)

       rand_int_0  ... rand_int_2
0               0  ...     190001
1               1  ...     190004
2               2  ...     190015
3               3  ...     190032
4               4  ...     190060
...           ...  ...        ...
40333           7  ...     149842
40334           8  ...     149844
40335           9  ...     149846
40336          10  ...     149855
40337          11  ...     149896

[40338 rows x 8 columns]


===================== B =====================

This section saves recipe names (index 'name') from the Pandas Dataframe to a csv file in the same Google Drive folder.

This file (total_name.csv) is a temporary dictionary for debugging topic modeling and LDA.

In [97]:
# extract the column of 'name' from the Pandas Dataframe
csv_name = xlsx_total_data[['name']]
# add an additional index column, in case that tokenization removes certain recipes
csv_name['index'] = csv_name.index

csv_path = '/'.join(XLSX_PATH.split(sep='/')[:-1]) + '/total_name.csv'
csv_name.to_csv(csv_path)

print(csv_name)

                                             name  index
0                             Winter Endive Salad      0
1                        Stuffed Artichoke Hearts      1
2                      Slow Cooker Moscow Chicken      2
3      Slow Cooker Creole Black Beans and Sausage      3
4              Champagne Sorbet with Berry Medley      4
...                                           ...    ...
40333                           Hawaiian Iced Tea  40333
40334                    Belgian Endive au Gratin  40334
40335                             Hot Cider Punch  40335
40336    Hash Brown Casserole for the Slow Cooker  40336
40337                       Samhain Pumpkin Bread  40337

[40338 rows x 2 columns]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


===================== C =====================

This section loads total_name.csv and saves a tokenized version of this csv back.

This file (token_name.csv) is used for topic modeling with LDA, and GloVe representation (optional).